In [58]:
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import networkx as nx

In [59]:
# Clase vértice - Usuários
class User:

  def __init__(self, user:str, email:str):

    """
    user: Usuário da rede
    email: email do usuário
    """
    self.user = user
    self.email = email
    self.hobbies = [] # Tags com interesses.

    
  def __str__(self): # Vai printar o objeto usuário como string
    return str(self.user)

  # Adiciona hobbies dos usuários  
  def addHobbie(self, hobbie):
    self.hobbies.append(hobbie)

In [60]:
# Classe Aresta - Conexões
class Conection:

  def __init__(self, user_1, user_2, n_interacoes):

    """
    User_1: Usuário seguidor
    User_2: Usuário que é seguido por User_1
    n_interacoes: Quantas vezes o usuário A curtiu, comentou ou compartilhou postagens do usuário B.
    """

    self.user_1 = user_1
    self.user_2 = user_2
    self.n_interacoes = n_interacoes

In [61]:
# Classe post
class Post:
    def __init__(self, id:int, author:User, content: str):
        self.id = id
        self.author = author
        self.content = content
        self.date = datetime.now()
        self.likes = set() # Usei o set para evitar duplicatas.

    def like(self, user: User):
        self.likes.add(user)

    def __str__(self):
        return f"Post {self.id} por {self.author.user}: {self.content}."

In [71]:
# Grafo - Unipoli
class UniPoli:

    def __init__(self):
        self.users = {}  # Dicionário para armazenar usuários
        self.conections = []  # Lista para armazenar conexões
        self.posts = []  # Lista para armazenar post

    # Adiciona usuários
    def addUser(self, user, email):
        if user not in self.users:
            self.users[user] = User(user, email)

    # Adiciona conexões - Seguir
    def addConection(self, user_1, user_2, n_interacoes):
        # Certifica-se de que os usuários existem antes de criar a conexão
        if user_1 in self.users and user_2 in self.users:
            a = self.users[user_1]
            b = self.users[user_2]
            conexao = Conection(a, b, n_interacoes)
            self.conections.append(conexao)
        else:
            print(f"Erro: Usuário '{user_1}' ou '{user_2}' não encontrado(s).")


    # Exclui usuário
    def delUser(self, user):
        if user in self.users:
            del self.users[user]
            # Apagar as arestas ligadas a user - filtro
            self.conections = [a for a in self.conections if a.user_1.user != user and a.user_2.user != user]
            print(f'Usuário {user} removido com sucesso.')
            return True
        else:
            print(f'Usuário {user} não encontrado.')
            return False

    # Exclui aresta - Deixar de seguir
    def delConection(self, a, b):
        if a in self.users and b in self.users:
            # Precisa iterar sobre uma cópia se for remover elementos
            original_connections = self.conections[:]
            found_connection = False
            for aresta in original_connections:
                if aresta.user_1.user == a and aresta.user_2.user == b:
                    self.conections.remove(aresta)
                    print(f'{a} deixou de seguir {b}.')
                    found_connection = True
                    return True # Conexão encontrada e removida
            if not found_connection:
                print(f'{a} não segue {b}.')
        else:
            print(f'{a} ou {b} não encontrado.')
        return False

    # Método postar
    def create_post(self, author_name, content: str):
        if author_name in self.users:
            author = self.users[author_name]
            post = Post(len(self.posts), author , content)
            self.posts.append(post)
            return post
        return None # Se autor não existe

    def feed(self, user_name: str):
        feed_posts = []
        if user_name not in self.users:
            return []

        current_user = self.users[user_name]

        # Econtrar todos os usuários que current_user segue
        user_following = set()
        for conection in self.conections:
            if conection.user_1 == current_user:
                user_following.add(conection.user_2)

        # Gerar o feed
        for post in self.posts:
            if post.author == current_user:
                feed_posts.append(post)
            elif post.author in user_following:
                feed_posts.append(post)
                
        # Ordena post pelos mais recentes
        return sorted(feed_posts, key=lambda p: p.date, reverse=True)
            
    # Mostrar grafo
    def display(self):
        print('\n=============================== UniPoli ================================\n')
        print('Usuários: ')
        for user_name in self.users: # Iterar por chaves (nomes de usuário)
            print(f'{user_name} ', end=' ')

        print('\n \nConexões: ')
        for connection in self.conections:
            print(f'{connection.user_1.user} segue {connection.user_2.user}. {connection.user_1.user} interagiu {connection.n_interacoes} vezes com o perfil de {connection.user_2.user}.\n')

        # Printar os posts
        print('\nPosts: ')
        for post in self.posts:
            print(f'Post #{post.id} de {post.author.user}: {post.content}. {post.date}\n')

    # Recomendação para um usuário específico
    def recommend(self, user_ref_name):
        print(f"\n--- Gerando recomendações para: {user_ref_name} ---")
        if user_ref_name not in self.users:
            print(f"ERRO RECOMEND: Usuário '{user_ref_name}' não encontrado.")
            return []

        # 1. Executar BFS para obter distâncias de todos os usuários a partir de user_ref
        _, distances_from_ref = self.bfs(user_ref_name)

        if distances_from_ref is None: # Se o BFS não conseguiu iniciar ou retornou erro
            print(f"Não foi possível calcular distâncias para recomendações de {user_ref_name}.")
            return []

        # 2. Identificar quem user_ref já segue (para não recomendar quem já é seguido)
        user_ref_obj = self.users[user_ref_name]
        already_following = set()
        for connection in self.conections:
            if connection.user_1 == user_ref_obj:
                already_following.add(connection.user_2.user) # Adiciona o NOME do usuário seguido

        recommended_users = []

        # 3. Filtrar usuários que estão a distância 2 e não são seguidos
        for user_name, distance in distances_from_ref.items():
            # Critérios para recomendação:
            # - Distância de 2 do user_ref_name
            # - Não é o próprio user_ref_name
            # - Não é alguém que user_ref_name já segue
            if distance == 2 and user_name != user_ref_name and user_name not in already_following:
                recommended_users.append(user_name)
        
        print(f"--- Recomendações para {user_ref_name}: {recommended_users} ---")
        return recommended_users

    # Atualiza o número de interações
    def updateLike(self, user_1_name, user_2_name): # Recebe nomes de usuário
        # Procura a aresta existente
        for aresta in self.conections:
            if aresta.user_1.user == user_1_name and aresta.user_2.user == user_2_name:
                aresta.n_interacoes += 1
                return True  # Atualizou com sucesso
        return False  # Usuário(s) ou conexão inexistente(s)

    def bfs(self, start_user_name: str):
        """
        Executa uma Busca em Largura (BFS) a partir de um usuário inicial.
        Retorna a ordem de visitação dos usuários (nomes) e suas distâncias do usuário inicial.
        """
        print(f"\n--- Iniciando BFS a partir de: {start_user_name} ---")
        if start_user_name not in self.users:
            print(f"ERRO BFS: Usuário '{start_user_name}' não encontrado.")
            return None, None # Retorna None para ordem e distâncias

        start_node = self.users[start_user_name]
        
        # visited: Guarda se um nó já foi visitado para evitar loops e re-visitas
        # As chaves são objetos User, os valores são booleanos
        visited = {user_obj: False for user_obj in self.users.values()}
        
        # distances: Guarda a distância do nó inicial para cada nó alcançável
        # As chaves são objetos User, os valores são inteiros
        distances = {user_obj: -1 for user_obj in self.users.values()}
        
        queue = [] # Fila para armazenar os nós a serem visitados (FIFO)
        traversal_order_names = [] # Lista para armazenar a ordem de visitação (nomes de usuário)

        # Configura o nó inicial
        queue.append(start_node)
        visited[start_node] = True
        distances[start_node] = 0

        while queue:
            current_node = queue.pop(0) # Remove o primeiro elemento da fila
            traversal_order_names.append(current_node.user) # Adiciona o nome do usuário à ordem de visitação
            
            print(f"  Visitando: {current_node.user} (Distância: {distances[current_node]})")

            # Percorre todos os vizinhos (usuários que current_node segue)
            for connection in self.conections:
                if connection.user_1 == current_node: # Se current_node segue connection.user_2
                    neighbor = connection.user_2
                    if not visited[neighbor]:
                        visited[neighbor] = True
                        distances[neighbor] = distances[current_node] + 1
                        queue.append(neighbor)
                        print(f"    Adicionado à fila: {neighbor.user}")
        
        # Formata o dicionário de distâncias para usar nomes de usuário como chaves
        # Apenas inclui usuários que foram visitados (distância != -1)
        formatted_distances = {user_obj.user: dist for user_obj, dist in distances.items() if dist != -1}
        
        print(f"--- BFS Concluída. Ordem: {traversal_order_names} ---")
        return traversal_order_names, formatted_distances

    def dfs(self, start_user_name: str):
        """
        Executa uma Busca em Profundidade (DFS) a partir de um usuário inicial.
        Retorna a ordem de visitação dos usuários (nomes).
        """
        print(f"\n--- Iniciando DFS a partir de: {start_user_name} ---")
        if start_user_name not in self.users:
            print(f"ERRO DFS: Usuário '{start_user_name}' não encontrado.")
            return None # Retorna None para a ordem

        start_node = self.users[start_user_name]

        # visited: Guarda se um nó já foi visitado
        visited = {user_obj: False for user_obj in self.users.values()}
        
        stack = [] # Pilha para armazenar os nós a serem visitados (LIFO)
        traversal_order_names = [] # Lista para armazenar a ordem de visitação (nomes de usuário)

        # Configura o nó inicial
        stack.append(start_node)
        
        while stack:
            current_node = stack.pop() # Remove o último elemento da pilha
            
            if not visited[current_node]:
                visited[current_node] = True
                traversal_order_names.append(current_node.user) # Adiciona o nome do usuário à ordem
                
                print(f"  Visitando: {current_node.user}")

                # Percorre todos os vizinhos (usuários que current_node segue)
                # Adiciona à pilha em ordem inversa para que o vizinho "mais à esquerda" seja visitado primeiro
                adjacents = []
                for connection in self.conections:
                    if connection.user_1 == current_node:
                        adjacents.append(connection.user_2)
                
                # Adiciona vizinhos à pilha (em ordem inversa para o comportamento correto de DFS)
                for neighbor in reversed(adjacents): # 'reversed' garante que a ordem de visita seja consistente
                    if not visited[neighbor]:
                        stack.append(neighbor)
                        print(f"    Adicionado à pilha: {neighbor.user}")
        
        print(f"--- DFS Concluída. Ordem: {traversal_order_names} ---")
        return traversal_order_names

In [73]:
## Teste
rede = UniPoli()

# Usuários
rede.addUser('Mikael', 'mikael@gmail.com')
rede.addUser('Madalena', 'Mad@gmail.com')
rede.addUser('Ana', 'Ana@gmail.com')
rede.addUser('Elon Musk', 'El@gmail.com')

# Conexões
rede.addConection('Mikael', 'Madalena', 1)
rede.addConection('Madalena', 'Mikael', 1)
rede.addConection('Madalena', 'Ana', 1)
rede.addConection('Ana', 'Madalena', 1)
rede.addConection('Mikael', 'Elon Musk', 100)

rede.display()

rede.updateLike('Mikael', 'Madalena')
rede.delConection('Mikael', 'Elon Musk')
rede.delConection('Mikael', 'Ana')
rede.delUser('Elon Musk')
rede.create_post('Mikael', 'Eu sou muito esperto!')
rede.create_post('Ana', 'Mikael é um gostoso!')
rede.create_post('Madalena', 'Mikael é super inteligente!')


rede.bfs('Ana')
rede.dfs('Ana')

rede.recommend('Mikael')

rede.display()

# Testar o feed
# Feed de Mikael
print("Feed de Mikael:")
for post in rede.feed('Mikael'):
    print(f"- {post}")

print('\n')
print("Feed de Madalena:")
for post in rede.feed('Madalena'):
    print(f"- {post}")


=============================== UniPoli ================================

Usuários: 
Mikael  Madalena  Ana  Elon Musk  
 
Conexões: 
Mikael segue Madalena. Mikael interagiu 1 vezes com o perfil de Madalena.

Madalena segue Mikael. Madalena interagiu 1 vezes com o perfil de Mikael.

Madalena segue Ana. Madalena interagiu 1 vezes com o perfil de Ana.

Ana segue Madalena. Ana interagiu 1 vezes com o perfil de Madalena.

Mikael segue Elon Musk. Mikael interagiu 100 vezes com o perfil de Elon Musk.


Posts: 
Mikael deixou de seguir Elon Musk.
Mikael não segue Ana.
Usuário Elon Musk removido com sucesso.

--- Gerando recomendações para: Mikael ---

--- Iniciando BFS a partir de: Mikael ---
  Visitando: Mikael (Distância: 0)
    Adicionado à fila: Madalena
  Visitando: Madalena (Distância: 1)
    Adicionado à fila: Ana
  Visitando: Ana (Distância: 2)
--- BFS Concluída. Ordem: ['Mikael', 'Madalena', 'Ana'] ---
--- Recomendações para Mikael: ['Ana'] ---

=============================== UniPoli

In [87]:
class Gui:
    def __init__(self, master):
        self.master = master
        master.title("UniPoli!")

        master.geometry('450x450')
        master.resizable(width=False, height=False)
        
        self.rede = UniPoli()

        # Dados de exemplo
        self.rede.addUser('Mikael', 'mikael@gmail.com')
        self.rede.addUser('Madalena', 'madalena@gmail.com')
        self.rede.addUser('Ana', 'ana@gmail.com')

        # Mikael segue Madalena
        self.rede.addConection('Mikael', 'Madalena', 1)
        
        # Madalena segue Mikael
        self.rede.addConection('Madalena', 'Mikael', 1)
        self.rede.addConection('Ana', 'Madalena', 1)
        self.rede.addConection('Madalena', 'Ana', 1)

        # Posts de Mikael
        self.rede.create_post('Mikael', 'Olá, UniPoli!')
        self.rede.create_post('Mikael', 'Primeiro post!')

        # Posts de Madalena 
        self.rede.create_post('Madalena', 'Que dia lindo em Recife!')
        self.rede.create_post('Madalena', 'Curtindo a paisagem.')

        # Posts de Ana 
        self.rede.create_post('Ana', 'Novo aqui na rede!')

        # Widgets da interface
        self.label = tk.Label(master, width=18, height=2, text="Bem-vindo UniPoli!", font=('Times', 18), fg='black')
        self.label.pack()

        # Botões, entradas de texto, etc.
        self.btn_sign_up = tk.Button(master, text="Ainda não tem conta: Sign up", command=self.open_add_user_window)
        self.btn_sign_up.pack(pady=10)

        self.btn_login = tk.Button(master, text='Sign in', command=self.open_login_window)
        self.btn_login.pack(pady=10)
        
        self.close_button = tk.Button(master, text="Sair", command=master.destroy)
        self.close_button.pack(pady=10)

    def open_add_user_window(self):
        # Janela para adicionar usuário
        add_user_window = tk.Toplevel(self.master)
        add_user_window.title("Adicionar Novo Usuário")

        add_user_window.geometry('450x450')
        add_user_window.resizable(width=False, height=False)

        tk.Label(add_user_window, text="Nome de Usuário:").pack()
        self.username_entry = tk.Entry(add_user_window)
        self.username_entry.pack()

        tk.Label(add_user_window, text="Email:").pack()
        self.email_entry = tk.Entry(add_user_window)
        self.email_entry.pack()

        tk.Button(add_user_window, text="Criar conta", command=self.add_user_action).pack()

    def add_user_action(self):
        try:
            username = self.username_entry.get()
            email = self.email_entry.get()

            if not username or not email:
                messagebox.showerror("Erro", "Nome de usuário e email não podem ser vazios.")
                return

            if username in self.rede.users:
                messagebox.showwarning("Aviso", f"Usuário '{username}' já existe.")
            else:
                self.rede.addUser(username, email)
                messagebox.showinfo("Sucesso", f"Usuário '{username}' adicionado com sucesso!")
                # Limpar campos após adicionar
                self.username_entry.delete(0, tk.END)
                self.email_entry.delete(0, tk.END)

        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro: {e}")

    def open_login_window(self):
        login_window = tk.Toplevel(self.master)
        login_window.title('Login')

        login_window.geometry('450x450')
        login_window.resizable(width=False, height=False)

        tk.Label(login_window, text="Nome de Usuário:").pack()
        self.name_entry = tk.Entry(login_window)
        self.name_entry.pack()

        tk.Label(login_window, text="Email:").pack()
        self.emailEntry = tk.Entry(login_window)
        self.emailEntry.pack()

        tk.Button(login_window, text="Entrar", command=self.open_main_window).pack()

    def open_main_window(self):

        try:
            user_name = self.name_entry.get()
            user_email = self.emailEntry.get()

            if not user_name or not user_email:
                messagebox.showerror("Erro", "Nome de usuário e email não podem ser vazios.")
                return

            if user_name not in self.rede.users:
                messagebox.showwarning("Aviso", f"Usuário '{user_name}' não tem conta.")
                return

            # Verifica e-mail
            current_user_obj = self.rede.users[user_name]
            if current_user_obj.email != user_email:
                messagebox.showerror("Erro", "Email incorreto para este usuário.")
                return

            # Limpar campos de login
            self.name_entry.delete(0, tk.END)
            self.emailEntry.delete(0, tk.END)

            # Criar a janela principal do usuário logado
            self.logged_in_user = user_name # Armazena o usuário logado para uso em outros métodos
            self.main_user_window = tk.Toplevel(self.master)
            self.main_user_window.title(f'UniPoli! - Logado como {user_name}')
            self.main_user_window.geometry('700x600') # Aumentei o tamanho para o feed
            self.main_user_window.resizable(width=False, height=True) # Pode redimensionar altura

            tk.Label(self.main_user_window, text=f"Bem-vindo, {user_name}!", font=('Times', 16)).pack(pady=10)

            # --- -----------------Adiciona área para o Feed ------------------------------
            tk.Label(self.main_user_window, text="Seu Feed:", font=('Times', 14, 'bold')).pack(pady=5)

            # Usar um Text widget para exibir o feed
            self.feed_text_area = tk.Text(self.main_user_window, wrap=tk.WORD, height=15, width=70)
            self.feed_text_area.pack(pady=5)
            
            # Adicionar um scrollbar para o Text widget
            scrollbar = tk.Scrollbar(self.main_user_window, command=self.feed_text_area.yview)
            scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
            self.feed_text_area.config(yscrollcommand=scrollbar.set)

            self.display_feed() # Chama o método para popular o feed inicialmente

            # Botões de funcionalidade na janela principal do usuário
            tk.Button(self.main_user_window, text="Atualizar Feed", command=self.display_feed).pack(pady=5)
            tk.Button(self.main_user_window, text="Criar Novo Post", command=self.open_create_post_window).pack(pady=5)
            tk.Button(self.main_user_window, text="Gerenciar Conexões", command=self.open_manage_connections_window).pack(pady=5)
            tk.Button(self.main_user_window, text="Mostrar Grafo (Console)", command=self.display_graph_in_console).pack(pady=5)
            tk.Button(self.main_user_window, text="Sair do Perfil", command=self.main_user_window.destroy).pack(pady=10)

            # Fecha a janela de login
            for widget in self.master.winfo_children():
                if isinstance(widget, tk.Toplevel) and widget.title() == 'Login':
                    widget.destroy()
                    break

                

        except Exception as e:
            messagebox.showerror("Erro", f"Ocorreu um erro: {e}")

        
    def display_graph_in_console(self):
        self.rede.display()
        messagebox.showinfo("Informação", "Detalhes do grafo foram impressos no console do Python.")

        
    def display_feed(self):
        # Limpa o conteúdo atual da área de texto do feed
        self.feed_text_area.delete(1.0, tk.END) # 1.0 significa da primeira linha, coluna 0 até o fim

        # Obtém o feed do usuário logado
        posts = self.rede.feed(self.logged_in_user) # Usa o usuário armazenado

        if not posts:
            self.feed_text_area.insert(tk.END, "Nenhum post no seu feed ainda.")
        else:
            for post in posts:
                self.feed_text_area.insert(tk.END, f"- Post #{post.id} de {post.author.user}: {post.content} (Likes: {len(post.likes)}) - {post.date.strftime('%H:%M %d/%m/%Y')}\n")

    def open_create_post_window(self):
        self.post_window = tk.Toplevel(self.main_user_window)
        self.post_window.geometry('450x450')
        self.post_window.resizable(width=False, height=False)
        self.post_window.title('Meus Posts')

        tk.Label(self.post_window, text=f'Vai postar o que hoje, {self.logged_in_user}?', font=('Times', 14, 'bold')).pack(pady=5)

        self.post_content = tk.Entry(self.post_window)
        self.post_content.pack(pady=5)

        tk.Button(self.post_window, text='Postar', command=self.post).pack(pady=10)

    # Cria o post
    def post(self):
        content = self.post_content.get()
        # Cria o post
        self.rede.create_post(self.logged_in_user, content)

        # Limpar campos de login
        self.post_content.delete(0, tk.END)
        
    def open_manage_connections_window(self):
        self.follow_window = tk.Toplevel(self.main_user_window)
        self.follow_window.title('Seguindo/Seguidores')
        self.follow_window.geometry('450x450')
        self.follow_window.resizable(width=False, height=True)

        tk.Label(self.follow_window, text='Pessoas que talvez você conheça', font=('Times', 14, 'bold')).pack(pady=5)

        # Usar um Text widget para exibir o feed
        self.feed_text_area2 = tk.Text(self.follow_window, wrap=tk.WORD, height=15, width=40)
        self.feed_text_area2.pack(pady=5)
        
        # Adicionar um scrollbar para o Text widget
        scrollbar = tk.Scrollbar(self.follow_window, command=self.feed_text_area2.yview)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
        self.feed_text_area2.config(yscrollcommand=scrollbar.set)

        # Adicionar usuários nos seguidores
        recommend = self.rede.recommend(self.logged_in_user)
        if len(recommend) == 0:
            self.feed_text_area2.insert(tk.END, "Nenhuma sugestão para você ainda.")
        else:
            for user in recommend:
                self.feed_text_area2.insert(tk.END, f"{user}\n")

        
        # Entry seguir
        self.follow_entry = tk.Entry(self.follow_window)
        self.follow_entry.pack(pady=5)

        # Botão seguir deixar de seguir
        tk.Button(self.follow_window, text="Seguir", command=self.follow).pack(pady=10)
        tk.Button(self.follow_window, text='Deixar de seguir', command=self.unfollow).pack(pady=10)

    def follow(self):
        follower = self.logged_in_user
        followed = self.follow_entry.get()
        self.rede.addConection(follower, followed, 1)
        self.display_feed()    
        self.follow_entry.delete(0, tk.END)

        
    def unfollow(self):
        user_follower = self.logged_in_user
        unfollowed = self.follow_entry.get()
        self.rede.delConection(user_follower, unfollowed)
        self.display_feed()
        self.follow_entry.delete(0, tk.END)

In [89]:
if __name__ == "__main__":
    root = tk.Tk()
    app = Gui(root)
    root.mainloop()


--- Gerando recomendações para: Bruno ---

--- Iniciando BFS a partir de: Bruno ---
  Visitando: Bruno (Distância: 0)
--- BFS Concluída. Ordem: ['Bruno'] ---
--- Recomendações para Bruno: [] ---

--- Gerando recomendações para: Mikael ---

--- Iniciando BFS a partir de: Mikael ---
  Visitando: Mikael (Distância: 0)
    Adicionado à fila: Madalena
  Visitando: Madalena (Distância: 1)
    Adicionado à fila: Ana
  Visitando: Ana (Distância: 2)
--- BFS Concluída. Ordem: ['Mikael', 'Madalena', 'Ana'] ---
--- Recomendações para Mikael: ['Ana'] ---

--- Gerando recomendações para: Mikael ---

--- Iniciando BFS a partir de: Mikael ---
  Visitando: Mikael (Distância: 0)
    Adicionado à fila: Madalena
    Adicionado à fila: Ana
  Visitando: Madalena (Distância: 1)
  Visitando: Ana (Distância: 1)
--- BFS Concluída. Ordem: ['Mikael', 'Madalena', 'Ana'] ---
--- Recomendações para Mikael: [] ---

--- Gerando recomendações para: Mikael ---

--- Iniciando BFS a partir de: Mikael ---
  Visitando: Mik